In [ ]:
import numpy as np
from collections import defaultdict,OrderedDict
import json
def stable_softmax(x):
    z = x - max(x)
    numerator = np.exp(z)
    denominator = np.sum(numerator)
    softmax = numerator/denominator

    return softmax

In [ ]:
file = 'big_data.json'
data = json.load(open(file,'rt',encoding='utf-8-sig'))

rt = ['stre', 'spd', 'jmp', 'endu', 'ins', 'dnk', 'ft', 'fg', 'tp', 'oiq', 'diq', 'drb', 'pss', 'reb', 'hgt']

In [ ]:
output = {'players':[],'teams':[]}

In [ ]:

college_teams = defaultdict(list)

for p in data['players']:
    if p['ratings'][0]['season'] != p['draft']['year']:
        continue
    
    pn = {}
    for t in ['firstName','lastName','college','hgt','weight','born']:
        pn[t] = p[t]
    pn['born']['year'] = 2020-(p['ratings'][0]['season']-p['born']['year'])
    pn['ratings'] = [p['ratings'][0]]
    pn['ratings'][0]['season'] = 2020
    
    clg = p['college']

    college_teams[clg].append(pn)
         
    

In [ ]:
tmp = sorted([(len(v),k) for k,v in college_teams.items()])[::-1]
tmp

In [ ]:
tot_p = sum([_[0] for _ in tmp])
frac = {v[1]:v[0]/tot_p for v in tmp}
frac

In [ ]:
sorted([(np.mean(sorted([_['ratings'][0]['ovr'] for _ in v])[-10:]),k) for k,v in college_teams.items() if len(v) >= 10])[::-1]

In [ ]:
teams = [k for k,v in college_teams.items() if len(v) >=10]

In [ ]:
def write_roman(num):

    roman = OrderedDict()
    roman[1000] = "M"
    roman[900] = "CM"
    roman[500] = "D"
    roman[400] = "CD"
    roman[100] = "C"
    roman[90] = "XC"
    roman[50] = "L"
    roman[40] = "XL"
    roman[10] = "X"
    roman[9] = "IX"
    roman[5] = "V"
    roman[4] = "IV"
    roman[1] = "I"

    def roman_num(num):
        for r in roman.keys():
            x, y = divmod(num, r)
            yield roman[r] * x
            num -= (r * x)
            if num <= 0:
                break

    return "".join([a for a in roman_num(num)])

In [ ]:
tm_lookup = {}
for i,t in enumerate(teams):
    if t == '':
        t = 'Undrafted'
    tm = {'tid':i,'cid':0,'did':0,'name':t,'abbrev':write_roman(i+1),'region':t}
    tm_lookup[t] = i
    output['teams'].append(tm)

In [ ]:
import random
i=0
ply_lookup = {}
ply_ratings = []

extra_players = []

for k,v in college_teams.items():
    if len(v) < 10:
        continue
    ply = sorted([(p['ratings'][0]['ovr'],random.random(),p) for p in v])
    pi = [j for j,_ in enumerate(ply)]
    p = stable_softmax(3e-2*np.array([_[0] for _ in ply]))
    
    idx = np.random.choice(pi,10,False)
    
    for j in idx:
        p = ply[j][2]
        p['tid'] = tm_lookup[k if k!= '' else 'Undrafted']
        p['pid'] = i
        ply_lookup[i] = p
        ply_ratings.append([p['ratings'][0][k] for k in rt])
        i+=1
        output['players'].append(p)
    for j in pi:
        if j in idx:
            continue
        extra_players.append(p)

In [ ]:
len(extra_players)

In [ ]:
with open('test_roster.json','wt') as fp:
    json.dump(output,fp)

In [ ]:
players = sum([[_['ratings'][0] for _ in v] for k,v in college_teams.items()],[])

In [ ]:
A = np.array([[_[k] for k in rt] for _ in players])

In [ ]:
N = 3
u,s,v = np.linalg.svd(A,full_matrices=False)
v[N:] = 0
#s[N:] = 1e-9
abs(A-(u@np.diag(s)@v)).mean()

In [ ]:
score = (A @ (v.T)@np.diag(1/s))

In [ ]:
data2 = json.load(open('realbad.json','rt',encoding='utf-8-sig'))

In [ ]:
printd = {}
tmp = ((v.T)@np.diag(1/s))[:,:3]
for name,row in zip(rt,tmp):
    printd[name] = list(row)
#print(json.dumps(printd,indent=True))

In [ ]:
stats_to_pred = ['pts','orb','drb','ast','stl','blk','min']
extra = ['per']

X = []
y = []
for p in data2['players']:
    try:
        f = frac[p['college']]
        age = 2020-p['born']['year']
        r = np.array([p['ratings'][0][k] for k in rt])
        pca = ((r @ v.T) @ (np.diag(1/s)))[:N]
        stats = p['stats'][0]
        if stats['min'] < 850:
            continue
        if stats['gs'] < 40:
            continue
        stats_v = [36*(stats[s]/stats['gp'])/(stats['min']/stats['gp']) for s in stats_to_pred] + [stats['per']]
        
        y.append(stats_v)

        X.append(list(pca))

    except:
        pass
X = np.array(X)
y = np.array(y)
X.shape,y.shape

In [ ]:
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import feature_selection, model_selection
from sklearn import  multioutput

import matplotlib.pyplot as plt

In [ ]:
fexp = preprocessing.PolynomialFeatures(degree=1,interaction_only=False)
scalerX = preprocessing.StandardScaler()
scalery = preprocessing.StandardScaler()

prescale_X = fexp.fit_transform(X)
prescale_X = scalerX.fit_transform(prescale_X)

prescale_y = scalery.fit_transform(y)

In [ ]:
clf = multioutput.MultiOutputRegressor(linear_model.LinearRegression())

In [ ]:
clf.fit(X,y)
res = clf.predict(X)
np.linalg.norm(res-np.array(y))

In [ ]:
clf.predict(np.array(  [ -0.005647817868185792, 0.005709440253439957, -0.0017545444845067665 ])[None,:])

In [ ]:
for s,cl in zip(stats_to_pred,clf.estimators_):
    print(s,':',list(np.round(list(cl.coef_)+[cl.intercept_],3)),',')

In [ ]:
plt.style.use('seaborn-white')
plt.figure(figsize=(8,8))
for i,s in enumerate(stats_to_pred + extra):
    plt.subplot(3,3,i+1)
    plt.title(s)
    plt.grid(True)
    plt.scatter(y[X[:,0] <= 1,i],res[X[:,0] <= 1,i],s=5,alpha=0.7)
    plt.scatter(y[X[:,0] > 1,i],res[X[:,0] > 1,i],s=15,alpha=0.7)

plt.suptitle('predicting college stats\n kinda',y=1.05,size=14)
plt.tight_layout()


In [ ]:
clf.estimators_[4].coef_.shape

In [ ]:
X.shape